### import CSV & query basic infomation
Include #event, #cases, the start activities and the end activities

In [61]:
import pandas as pd
import pm4py as pm

training_file_path = '12689204\BPI_Challenge_2012.XE\BPI_Challenge_2012.XE-training.csv'




In [62]:
event_log = pd.read_csv(training_file_path, sep=',')
event_log = pm.format_dataframe(
    event_log, 
    case_id='case concept:name', 
    activity_key='event concept:name', 
    timestamp_key='event time:timestamp'
)
num_events = len(event_log)
num_cases = len(event_log['case concept:name'].unique())
print("Number of events: {}\nNumber of cases: {}".format(num_events, num_cases))
print('=======================================================')

start_activities = pm.get_start_activities(event_log)
end_activities = pm.get_end_activities(event_log)
print("Start activities: {}\nEnd activities: {}".format(start_activities, end_activities))
print('=======================================================')

print('events: ')
print(event_log['event concept:name'].unique())

C:\Users\Yu-Hsuan Lin\AppData\Local\Temp\ipykernel_9632\3438941301.py:2: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  event_log = pm.format_dataframe(


Number of events: 239786
Number of cases: 12478
Start activities: {'A_SUBMITTED': 10097, 'W_Nabellen offertes': 1484, 'W_Completeren aanvraag': 391, 'W_Valideren aanvraag': 299, 'W_Nabellen incomplete dossiers': 124, 'W_Afhandelen leads': 65, 'O_CANCELLED': 9, 'W_Beoordelen fraude': 6, 'A_CANCELLED': 3}
End activities: {'A_DECLINED': 3429, 'W_Completeren aanvraag': 2428, 'W_Afhandelen leads': 2269, 'W_Nabellen offertes': 2033, 'W_Valideren aanvraag': 1417, 'W_Nabellen incomplete dossiers': 399, 'A_CANCELLED': 324, 'O_CANCELLED': 117, 'W_Beoordelen fraude': 59, 'W_Wijzigen contractgegevens': 3}
events: 
['A_SUBMITTED' 'A_PARTLYSUBMITTED' 'A_PREACCEPTED'
 'W_Completeren aanvraag' 'A_ACCEPTED' 'A_FINALIZED' 'O_SELECTED'
 'O_CREATED' 'O_SENT' 'W_Nabellen offertes' 'O_SENT_BACK'
 'W_Valideren aanvraag' 'O_ACCEPTED' 'A_REGISTERED' 'A_APPROVED'
 'A_ACTIVATED' 'O_CANCELLED' 'W_Wijzigen contractgegevens' 'A_DECLINED'
 'A_CANCELLED' 'W_Afhandelen leads' 'O_DECLINED'
 'W_Nabellen incomplete dossi

In [63]:
test_time_range_filtered_log = pm.filter_time_range(event_log, "2011-01-01 00:00:00", "2013-01-01 00:00:00", mode='events')
# print(test_time_range_filtered_log)



In [71]:


def getCase(df,case_ID):
    df_case = df[df['case concept:name'] == case_ID]
    return df_case


    
def getNextEventIndex(df_case, currentEventID):
    df = df_case.sort_values(by='eventID')
    token = False
    for index, r in df.iterrows():
        if token:
            return index
        if currentEventID == df['eventID'][index]:
            token = True
    return "N\A"
    
def writeGroundtruth(df):
    df = df.head(1000)
    df = df.assign(ground_truth_activity='')
    df = df.assign(ground_truth_time='')

    for ind in df.index:
        case_num = df['case concept:name'][ind]
        df_case = getCase(df, case_num)
        next_event_index = getNextEventIndex(df_case, df_case['eventID'][ind])
        if next_event_index != "N\A":
            df.at[ind,'ground_truth_activity'] = df.loc[next_event_index,'event concept:name']
            df.at[ind,'ground_truth_time'] = df.loc[next_event_index, 'event time:timestamp']
        else:
            df.at[ind,'ground_truth_activity'] = "N\A"
            df.at[ind,'ground_truth_time'] = "N\A"

    return df


In [72]:

df_event = writeGroundtruth(event_log)
df_event

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp,case:concept:name,concept:name,time:timestamp,@@index,@@case_index,ground_truth_activity,ground_truth_time
0,0,173688,2011-01-10 00:38:44.546000+00:00,20000,112.0,A_SUBMITTED,COMPLETE,01-10-2011 00:38:44.546,173688,A_SUBMITTED,2011-01-10 00:38:44.546000+00:00,0,0,A_PARTLYSUBMITTED,01-10-2011 00:38:44.880
1,1,173688,2011-01-10 00:38:44.546000+00:00,20000,112.0,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 00:38:44.880,173688,A_PARTLYSUBMITTED,2011-01-10 00:38:44.880000+00:00,1,0,A_PREACCEPTED,01-10-2011 00:39:37.906
2,2,173688,2011-01-10 00:38:44.546000+00:00,20000,112.0,A_PREACCEPTED,COMPLETE,01-10-2011 00:39:37.906,173688,A_PREACCEPTED,2011-01-10 00:39:37.906000+00:00,2,0,W_Completeren aanvraag,01-10-2011 00:39:38.875
3,3,173688,2011-01-10 00:38:44.546000+00:00,20000,112.0,W_Completeren aanvraag,SCHEDULE,01-10-2011 00:39:38.875,173688,W_Completeren aanvraag,2011-01-10 00:39:38.875000+00:00,3,0,W_Completeren aanvraag,01-10-2011 11:36:46.437
4,4,173688,2011-01-10 00:38:44.546000+00:00,20000,NaN,W_Completeren aanvraag,START,01-10-2011 11:36:46.437,173688,W_Completeren aanvraag,2011-01-10 11:36:46.437000+00:00,4,0,A_ACCEPTED,01-10-2011 11:42:43.308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,154618822666,173796,2011-01-10 20:19:11.828000+00:00,15000,11001.0,O_CREATED,COMPLETE,03-10-2011 11:16:42.008,173796,O_CREATED,2011-03-10 11:16:42.008000+00:00,995,36,O_SENT,03-10-2011 11:16:42.037
996,154618822667,173796,2011-01-10 20:19:11.828000+00:00,15000,11001.0,O_SENT,COMPLETE,03-10-2011 11:16:42.037,173796,O_SENT,2011-03-10 11:16:42.037000+00:00,996,36,W_Nabellen offertes,03-10-2011 11:16:42.193
997,154618822668,173796,2011-01-10 20:19:11.828000+00:00,15000,11001.0,W_Nabellen offertes,SCHEDULE,03-10-2011 11:16:42.193,173796,W_Nabellen offertes,2011-03-10 11:16:42.193000+00:00,997,36,W_Completeren aanvraag,03-10-2011 11:16:43.451
998,154618822669,173796,2011-01-10 20:19:11.828000+00:00,15000,11001.0,W_Completeren aanvraag,COMPLETE,03-10-2011 11:16:43.451,173796,W_Completeren aanvraag,2011-03-10 11:16:43.451000+00:00,998,36,W_Nabellen offertes,10-10-2011 17:23:25.455
